In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import gc

C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
df=pd.read_excel("datathon.xlsx",skiprows=3)

In [5]:
df.head(10)

,Cluster,Brand Group,Country,Function,Jan 2012,Feb 2012,Mar 2012,Unnamed: 7,Apr 2012,May 2012,...,Unnamed: 113,Jul 2018,Aug 2018,Sep 2018,Unnamed: 117,Oct 2018,Nov 2018,Dec 2018,Unnamed: 121,Unnamed: 122
0,Cluster 3,Brand Group 12,Country 19,Sales 1,1861.328751,1995.945017,1998.533792,5855.807560,2019.243986,2425.681558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cluster 3,Brand Group 12,Country 19,Sales 2,1316.247027,1450.223283,1440.837462,4207.307772,1564.066421,2014.132904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster 3,Brand Group 12,Country 19,Investment 1,-216.162658,-238.167239,-217.457045,-671.786942,-214.868270,-390.904926,...,-431.807560,-146.395189,-145.100802,-171.894616,-463.390607,-136.946163,-189.974966,-156.680908,-483.602037,-1816.303068
3,Cluster 3,Brand Group 12,Country 19,Investment 2,-107.434135,-14.238259,-62.130584,-183.802978,-91.901489,-199.335624,...,-155.455899,-35.595647,-64.460481,-111.705613,-211.761741,-29.253150,-133.434491,-88.018327,-250.705968,-696.492909
4,Cluster 3,Brand Group 12,Country 19,Investment 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.106529,-3.106529,0.000000,0.000000,0.000000,0.000000,-1.423826
5,Cluster 3,Brand Group 12,Country 19,Investment 6,-56.953036,-55.658648,-64.719359,-177.331042,-56.953036,-55.658648,...,-149.890034,-42.197022,-41.032073,-48.539519,-131.768614,-45.044674,-45.007720,-35.730998,-125.783392,-544.129325
6,Cluster 3,Brand Group 17,Country 19,Sales 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cluster 3,Brand Group 17,Country 19,Sales 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cluster 3,Brand Group 17,Country 19,Investment 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-6300.688431,-2123.312715,-1912.457045,-2342.970218,-6378.739977,-2114.899198,-2206.086863,-1968.091008,-6289.077068,-24996.207653
9,Cluster 3,Brand Group 17,Country 19,Investment 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-7063.470791,-1790.266896,-1048.194731,-1904.690722,-4743.152348,-1511.585338,-1595.179450,-1041.791168,-4148.555956,-19825.655613


##  Sales1

In [6]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
dfsales2=df.loc[df['Function'] == "Sales 2"]
dfsales2 = dfsales2[dfsales2.columns.drop(list(dfsales2.filter(regex='2018')))]
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')
dfmelt=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value_Y")

In [7]:
dfmelt.head(5)

,Cluster,Brand Group,Date,Value_Y
0,Cluster 1,Brand Group 12,Jan 2012,616.408961
1,Cluster 1,Brand Group 13,Jan 2012,62.381470
2,Cluster 1,Brand Group 15,Jan 2012,0.000000
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992
4,Cluster 1,Brand Group 17,Jan 2012,0.000000


## Sales 2

In [8]:
dfinves=df.loc[df['Function'].str.contains("Investment")]

In [1503]:
dfinves=dfinves.groupby(['Cluster','Brand Group','Function']).agg('sum')


In [1484]:
dfinves=pd.melt(dfinves.reset_index(), id_vars=["Cluster", "Brand Group","Function"], 
                  var_name="Date", value_name="Investment")

In [1485]:
dfinves=dfinves.pivot_table('Investment', ['Cluster', 'Brand Group','Date'], 'Function')

In [1486]:
dfinves=dfinves.reset_index()

In [1487]:
dfinves=dfinves.fillna(0)

In [1488]:
dfinves.head(5)

Function,Cluster,Brand Group,Date,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,Apr 2012,-39.664141,-79.814354,0.000000,-1.451180,0.0,-9.745249
1,Cluster 1,Brand Group 12,Apr 2013,-44.510635,-144.391369,0.000000,-1.462129,0.0,-12.250295
2,Cluster 1,Brand Group 12,Apr 2014,-51.325172,-118.808299,-0.682549,-3.965413,0.0,-11.078440
3,Cluster 1,Brand Group 12,Apr 2015,-47.388726,-62.490278,0.000000,-3.320586,0.0,-14.386244
4,Cluster 1,Brand Group 12,Apr 2016,-55.299492,-115.182981,0.000000,-8.180448,0.0,-7.625617


In [1489]:
dfinves=dfinves.replace({"Date": dictmonths})

In [1490]:
dfinves.columns=['Cluster', 'Brand Group', 'Date_coded', 'Investment 1', 'Investment 2',
       'Investment 3', 'Investment 4', 'Investment 5', 'Investment 6']

In [1491]:
dfmerged=dfmerged.fillna(0)

In [1492]:
dfmerged=pd.merge(dfmelt,dfinves,
                 on=['Cluster','Brand Group','Date'])

KeyError: 'Date'

In [ ]:
dfmerged.head(5)

In [1504]:
dictmonths=dict(zip(dfmelt["Date"].unique(),range(len(dfmelt["Date"].unique()))))

In [1505]:
dfmelt["Date_coded"]=dfmelt["Date"].apply(lambda x: dictmonths[x])

In [1506]:
dfmelt["Month"]=dfmelt["Date_coded"].apply(lambda x: x%12)

In [1508]:
import numpy as np
dfmelt["Actual"] = np.nan
dfmelt["Lag-1"] = np.nan
dfmelt["Lag-2"] = np.nan
dfmelt["Lag-3"] = np.nan
# dfmelt["Lag-4"] = np.nan

In [1509]:
dfmelt["Lag-4"] = np.nan
dfmelt["Lag-5"] = np.nan
dfmelt["Lag-6"] = np.nan
dfmelt["Lag-7"] = np.nan
dfmelt["Lag-8"] = np.nan
dfmelt["Lag-9"] = np.nan
dfmelt["Lag-10"] = np.nan
dfmelt["Lag-11"] = np.nan
dfmelt["Lag-12"] = np.nan

In [1568]:
dfpre[(dfpre["Cluster"]=="Cluster 9") 
                   & (dfpre["Brand Group"]=="Brand Group 97")
                                                                &(dfpre["Date_coded"]==71-12)]

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,Lag-4,Lag-5,Lag-6,Lag-7,Lag-8,Lag-9,Lag-10,Lag-11,Lag-12
18179,Cluster 9,Brand Group 97,Dec 2016,3805.376713,59,11,3544.962927,3217.108722,3447.37032,3490.445114,3110.535949,3627.127041,3181.665455,3182.77481,3479.391416,3216.889196,2717.86426,3364.681084,NaN


In [1547]:
def add_lag_values(dfpre,lag,lagname):
    for i in range (len (dfpre)):
        if dfpre.iloc[i].Date_coded>12:
            dfpre.iloc[i,dfpre.columns.get_loc(lagname)]=dfpre[(dfpre["Cluster"]==dfpre.iloc[i].Cluster) 
                   & (dfpre["Brand Group"]==dfpre.iloc[i,dfpre.columns.get_loc("Brand Group")])
                                                                &(dfpre["Date_coded"]==dfpre.iloc[i].Date_coded-lag)]["Value_Y"].values[0]
    return dfpre

In [1557]:
dfpre

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,Lag-4,Lag-5,Lag-6,Lag-7,Lag-8,Lag-9,Lag-10,Lag-11,Lag-12
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cluster 1,Brand Group 13,Jan 2012,62.381470,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster 1,Brand Group 15,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cluster 1,Brand Group 17,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cluster 1,Brand Group 18,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cluster 1,Brand Group 19,Jan 2012,1328.072408,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cluster 1,Brand Group 24,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cluster 1,Brand Group 25,Jan 2012,1869.355444,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Cluster 1,Brand Group 26,Jan 2012,2044.415276,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1512]:
dfpre=add_lag_values(dfmelt,1,'Actual')
dfpre=add_lag_values(dfpre,2,'Lag-1')
dfpre=add_lag_values(dfpre,3,'Lag-2')
dfpre=add_lag_values(dfpre,4,'Lag-3')

In [1513]:
dfpre=add_lag_values(dfpre,5,'Lag-4')
dfpre=add_lag_values(dfpre,6,'Lag-5')
dfpre=add_lag_values(dfpre,7,'Lag-6')
dfpre=add_lag_values(dfpre,8,'Lag-7')
dfpre=add_lag_values(dfpre,9,'Lag-8')
dfpre=add_lag_values(dfpre,10,'Lag-9')
dfpre=add_lag_values(dfpre,11,'Lag-10')
dfpre=add_lag_values(dfpre,12,'Lag-11')

In [1565]:
dfpre=add_lag_values(dfpre,12,'Lag-11')

In [1569]:
del dfpre["Lag-12"]

In [1570]:
#dfpre.to_pickle("lags_pre_12")

In [437]:
dfpre=pd.read_pickle("lags_pre_12")

In [438]:
dfpre=dfpre.dropna()

In [439]:
dfpre['Month']=dfpre['Month'].apply(lambda x: 11 if x == 0 else x-1)

In [440]:
dfpre.head(5)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,Lag-4,Lag-5,Lag-6,Lag-7,Lag-8,Lag-9,Lag-10,Lag-11
3939,Cluster 1,Brand Group 12,Feb 2013,886.812584,13,0,745.704077,1084.703889,946.821916,943.346418,837.083837,658.304347,953.191022,319.401731,656.607506,723.701936,968.938870,685.081035
3940,Cluster 1,Brand Group 13,Feb 2013,32.448225,13,0,47.174759,46.998244,80.600039,58.313315,49.465439,62.483972,46.670999,36.194606,65.677360,55.139918,42.081080,69.445132
3941,Cluster 1,Brand Group 15,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3942,Cluster 1,Brand Group 16,Feb 2013,2297.069386,13,0,1965.631205,2106.791079,2956.718563,2495.493383,2002.401507,2284.694096,2335.718836,1909.803910,2341.879305,2521.239601,2019.736343,2616.171336
3943,Cluster 1,Brand Group 17,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [441]:
dfpre['sales_trend'] = dfpre['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

In [442]:
dfpre['mid_sales_trend'] = dfpre['Actual']

lag_list=[1,2,3,4,5,6]

for lag in lag_list:
    ft_name = ('Lag-%s' % lag)
    dfpre['mid_sales_trend'] -= dfpre[ft_name]

dfpre['mid_sales_trend'] /= len(lag_list) + 1

In [443]:
# dfpre['long_sales_trend'] = dfpre['Actual']

# lag_list=[1,2,3,4,5,6,7,8,9,10,11]

# for lag in lag_list:
#     ft_name = ('Lag-%s' % lag)
#     dfpre['long_sales_trend'] -= dfpre[ft_name]

# dfpre['long_sales_trend'] /= len(lag_list) + 1

In [444]:
dfpre=dfpre.dropna()

In [445]:
dfpre

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,Lag-4,Lag-5,Lag-6,Lag-7,Lag-8,Lag-9,Lag-10,Lag-11,sales_trend,mid_sales_trend
3939,Cluster 1,Brand Group 12,Feb 2013,886.812584,13,0,745.704077,1084.703889,946.821916,943.346418,837.083837,658.304347,953.191022,319.401731,656.607506,723.701936,968.938870,685.081035,-557.292036,-668.249622
3940,Cluster 1,Brand Group 13,Feb 2013,32.448225,13,0,47.174759,46.998244,80.600039,58.313315,49.465439,62.483972,46.670999,36.194606,65.677360,55.139918,42.081080,69.445132,-34.684210,-42.479607
3941,Cluster 1,Brand Group 15,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3942,Cluster 1,Brand Group 16,Feb 2013,2297.069386,13,0,1965.631205,2106.791079,2956.718563,2495.493383,2002.401507,2284.694096,2335.718836,1909.803910,2341.879305,2521.239601,2019.736343,2616.171336,-1398.342955,-1745.169465
3943,Cluster 1,Brand Group 17,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3944,Cluster 1,Brand Group 18,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3945,Cluster 1,Brand Group 19,Feb 2013,1498.933024,13,0,1256.505397,1204.286111,1735.955221,1139.976953,1604.710565,1770.380294,1610.354841,2047.555687,1691.095349,2039.346028,1843.272270,1747.087079,-705.928222,-1115.594084
3946,Cluster 1,Brand Group 24,Feb 2013,0.000000,13,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3947,Cluster 1,Brand Group 25,Feb 2013,2398.767022,13,0,2229.490821,2138.460934,2336.864699,3205.431354,2741.366735,2329.189122,2453.273551,1882.068631,1714.321134,2478.010511,3067.466622,1724.577575,-1362.816541,-1853.585082
3948,Cluster 1,Brand Group 26,Feb 2013,2710.350144,13,0,2550.119528,1905.375056,1800.682066,2760.493348,1762.209452,2267.608240,2335.513978,2218.761723,2528.122320,2272.917754,2773.263487,2124.060822,-979.107736,-1468.823230


In [446]:
dfpre['Date_coded']=dfpre['Date_coded'].apply(lambda x: x-1)

In [447]:
dfinves.head(5)

,Cluster,Brand Group,Country,Function,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,...,Mar 2018,Apr 2018,May 2018,Jun 2018,Jul 2018,Aug 2018,Sep 2018,Oct 2018,Nov 2018,Dec 2018
2,Cluster 3,Brand Group 12,Country 19,Investment 1,-216.162658,-238.167239,-217.457045,-214.868270,-390.904926,-120.378007,...,-159.727377,-141.994273,-127.367698,-162.445590,-146.395189,-145.100802,-171.894616,-136.946163,-189.974966,-156.680908
3,Cluster 3,Brand Group 12,Country 19,Investment 2,-107.434135,-14.238259,-62.130584,-91.901489,-199.335624,-253.699885,...,-43.361970,-83.487973,-34.560137,-37.407789,-35.595647,-64.460481,-111.705613,-29.253150,-133.434491,-88.018327
4,Cluster 3,Brand Group 12,Country 19,Investment 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.682703,0.000000,0.000000,0.000000,0.000000,0.000000,-3.106529,0.000000,0.000000,0.000000
5,Cluster 3,Brand Group 12,Country 19,Investment 6,-56.953036,-55.658648,-64.719359,-56.953036,-55.658648,-93.195876,...,-47.115693,-49.445590,-44.268041,-56.176403,-42.197022,-41.032073,-48.539519,-45.044674,-45.007720,-35.730998
8,Cluster 3,Brand Group 17,Country 19,Investment 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2105.967927,-1952.841924,-2053.674685,-2294.171821,-2123.312715,-1912.457045,-2342.970218,-2114.899198,-2206.086863,-1968.091008


In [296]:
dffinal=pd.merge(dfpre_sales,dfinves,
                 on=['Cluster','Brand Group','Date_coded'])

NameError: name 'dfpre_sales' is not defined

In [132]:
dffinal.head(10)

NameError: name 'dffinal' is not defined

In [133]:
#dfpre.to_pickle("dataset_1_08.pkl")

In [134]:
#dfpre= pd.read_pickle("dataset_1_08.pkl")

In [448]:
yvalues =dfpre["Value_Y"]
xvalues = dfpre.drop(labels=['Value_Y'],axis=1)

In [449]:
from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
##sc.fit(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])
#xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [450]:
dfpre

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,Lag-4,Lag-5,Lag-6,Lag-7,Lag-8,Lag-9,Lag-10,Lag-11,sales_trend,mid_sales_trend
3939,Cluster 1,Brand Group 12,Feb 2013,886.812584,12,0,745.704077,1084.703889,946.821916,943.346418,837.083837,658.304347,953.191022,319.401731,656.607506,723.701936,968.938870,685.081035,-557.292036,-668.249622
3940,Cluster 1,Brand Group 13,Feb 2013,32.448225,12,0,47.174759,46.998244,80.600039,58.313315,49.465439,62.483972,46.670999,36.194606,65.677360,55.139918,42.081080,69.445132,-34.684210,-42.479607
3941,Cluster 1,Brand Group 15,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3942,Cluster 1,Brand Group 16,Feb 2013,2297.069386,12,0,1965.631205,2106.791079,2956.718563,2495.493383,2002.401507,2284.694096,2335.718836,1909.803910,2341.879305,2521.239601,2019.736343,2616.171336,-1398.342955,-1745.169465
3943,Cluster 1,Brand Group 17,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3944,Cluster 1,Brand Group 18,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3945,Cluster 1,Brand Group 19,Feb 2013,1498.933024,12,0,1256.505397,1204.286111,1735.955221,1139.976953,1604.710565,1770.380294,1610.354841,2047.555687,1691.095349,2039.346028,1843.272270,1747.087079,-705.928222,-1115.594084
3946,Cluster 1,Brand Group 24,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3947,Cluster 1,Brand Group 25,Feb 2013,2398.767022,12,0,2229.490821,2138.460934,2336.864699,3205.431354,2741.366735,2329.189122,2453.273551,1882.068631,1714.321134,2478.010511,3067.466622,1724.577575,-1362.816541,-1853.585082
3948,Cluster 1,Brand Group 26,Feb 2013,2710.350144,12,0,2550.119528,1905.375056,1800.682066,2760.493348,1762.209452,2267.608240,2335.513978,2218.761723,2528.122320,2272.917754,2773.263487,2124.060822,-979.107736,-1468.823230


In [451]:
X_train, X_valid, y_train, y_valid = train_test_split(xvalues, yvalues, test_size=0.15, shuffle=False)

In [452]:

xvalues.columns

Index(['Cluster', 'Brand Group', 'Date', 'Date_coded', 'Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'Lag-4', 'Lag-5', 'Lag-6', 'Lag-7', 'Lag-8',
       'Lag-9', 'Lag-10', 'Lag-11', 'sales_trend', 'mid_sales_trend'],
      dtype='object')

In [453]:
invest=['Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6']

In [454]:
X_train=X_train[['Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'Lag-4', 'Lag-5', 'Lag-6', 'sales_trend','mid_sales_trend']]
X_valid=X_valid[['Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'Lag-4', 'Lag-5', 'Lag-6', 'sales_trend','mid_sales_trend']]

In [455]:
models=[]

In [456]:
from xgboost import XGBRegressor
from sklearn.svm import SVR

In [457]:
regres=SVR(kernel = 'rbf',C=10000,gamma=.000000009)
regres.fit(xvalues[['Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'Lag-4', 'Lag-5', 'Lag-6', 'sales_trend','mid_sales_trend']],yvalues)

SVR(C=10000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=9e-09,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [458]:
models.append(regres)

In [459]:
predTest=regres.predict(X_valid)

In [460]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print("MAE:",mean_absolute_error(y_valid, predTest))
print("MSE:",mean_squared_error(y_valid, predTest))

MAE: 224.24197920037216
MSE: 410509.59927660855


In [306]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestRegressor
from mlxtend.classifier import StackingClassifier
import numpy as np

clf1 = KNeighborsRegressor(n_neighbors=1)
clf2 = RandomForestRegressor(random_state=1)
clf3 = SVR(kernel = 'rbf',C=10000,gamma=.000000009)
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X_train, y_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:



ValueError: continuous is not supported

In [1375]:
# Various hyper-parameters to tune
from sklearn.model_selection import GridSearchCV
model = XGBRegressor(eval_metric = 'rmse', early_stopping_rounds = 20)
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [6, 7],
              'min_child_weight': [15,20],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.8],
              'n_estimators': [1000]}

xgb_grid = GridSearchCV(model,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train,  eval_metric="rmse")

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.2min
[Parallel(n_jobs=5)]: Done  54 out of  54 | elapsed:  1.6min finished


0.9657134384098217
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 15, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [462]:
model = XGBRegressor(
    max_depth=6,
    learning_rate=0.03,
    n_estimators=500,
    min_child_weight=17, 
    colsample_bytree=0.7, 
    subsample=0.7, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
  
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=True, 
    early_stopping_rounds = 30)

[0]	validation_0-rmse:3971.41	validation_1-rmse:4556.44
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 30 rounds.
[1]	validation_0-rmse:3857.72	validation_1-rmse:4426.8
[2]	validation_0-rmse:3747.34	validation_1-rmse:4299.28
[3]	validation_0-rmse:3640.41	validation_1-rmse:4176.96
[4]	validation_0-rmse:3537.05	validation_1-rmse:4058.99
[5]	validation_0-rmse:3436.64	validation_1-rmse:3945.01
[6]	validation_0-rmse:3339.5	validation_1-rmse:3833.13
[7]	validation_0-rmse:3245.17	validation_1-rmse:3723.63
[8]	validation_0-rmse:3153.01	validation_1-rmse:3618.83
[9]	validation_0-rmse:3063.74	validation_1-rmse:3516.3
[10]	validation_0-rmse:2977.75	validation_1-rmse:3417.72
[11]	validation_0-rmse:2894.69	validation_1-rmse:3323.19
[12]	validation_0-rmse:2814.04	validation_1-rmse:3230.9
[13]	validation_0-rmse:2735.22	validation_1-rmse:3141.07
[14]	validation_0-rmse:2659.08	validation_1-rmse:3054.07


[141]	validation_0-rmse:528.744	validation_1-rmse:764.686
[142]	validation_0-rmse:528.127	validation_1-rmse:764.528
[143]	validation_0-rmse:527.618	validation_1-rmse:764.402
[144]	validation_0-rmse:526.985	validation_1-rmse:764.18
[145]	validation_0-rmse:526.439	validation_1-rmse:764.069
[146]	validation_0-rmse:525.918	validation_1-rmse:764.339
[147]	validation_0-rmse:525.221	validation_1-rmse:764.25
[148]	validation_0-rmse:524.712	validation_1-rmse:764.635
[149]	validation_0-rmse:524.288	validation_1-rmse:764.806
[150]	validation_0-rmse:523.805	validation_1-rmse:764.77
[151]	validation_0-rmse:523.35	validation_1-rmse:764.845
[152]	validation_0-rmse:523.145	validation_1-rmse:764.883
[153]	validation_0-rmse:522.774	validation_1-rmse:764.894
[154]	validation_0-rmse:521.927	validation_1-rmse:764.633
[155]	validation_0-rmse:521.612	validation_1-rmse:764.881
[156]	validation_0-rmse:520.719	validation_1-rmse:764.574
[157]	validation_0-rmse:520.334	validation_1-rmse:764.64
[158]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eta=0.3, gamma=0, learning_rate=0.03,
       max_delta_step=0, max_depth=6, min_child_weight=17, missing=None,
       n_estimators=500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.7)

In [311]:
models.append(model)

In [1416]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=14, random_state=0, n_estimators=75)
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [1417]:
p = regr.predict(X_valid)
print("MAE:",mean_absolute_error(y_valid, p))
print("MSE:",mean_squared_error(y_valid, p))

MAE: 270.43729730640075
MSE: 471365.5167566193


In [1429]:
from lightgbm import LGBMRegressor
mdl = LGBMRegressor( task= 'train',boosting_type= 'gbdt',objective= 'regression',num_leaves= 300, learning_rate= 0.32,feature_fraction= 0.9,
                    bagging_fraction= .9,bagging_freq= 70,verbose= 100)
mdl.fit(X_train, np.log1p(y_train))

C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: invalid value encountered in log1p


LGBMRegressor(bagging_fraction=0.9, bagging_freq=70, boosting_type='gbdt',
       class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
       importance_type='split', learning_rate=0.32, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=300, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
       task='train', verbose=100)

In [1434]:
p = mdl.predict(X_valid)
print("MAE:",mean_absolute_error(y_valid, p))
print("MSE:",mean_squared_error(y_valid, p))

MAE: 1933.2745963739865
MSE: 19682032.82794994


# Pred dataset

In [463]:
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')

In [464]:
dfsales2

Jan 2012      Feb 2012      Mar 2012  \
Cluster   Brand Group                                                
Cluster 1 Brand Group 12    616.408961    685.081035    968.938870   
          Brand Group 13     62.381470     69.445132     42.081080   
          Brand Group 15      0.000000      0.000000      0.000000   
          Brand Group 16   2457.695992   2616.171336   2019.736343   
          Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 18      0.000000      0.000000      0.000000   
          Brand Group 19   1328.072408   1747.087079   1843.272270   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1869.355444   1724.577575   3067.466622   
          Brand Group 26   2044.415276   2124.060822   2773.263487   
          Brand Group 27    616.696052    943.158430    991.609400   
          Brand Group 28      0.000000      0.000000      0.000000   
          Brand Group 30   1451.344742   2421.703682   2174.680037   
          Brand Group 31    601.508882   1221.516101    330.240589   
          Brand Group 36    224.091013     10.742677      3.547430   
          Brand Group 38    158.842381    140.429788    136.151464   
          Brand Group 39    667.435844    696.507436    618.169416   
          Brand Group 40    203.767645    186.701557    136.532992   
          Brand Group 41   2427.396563   3221.070147   3093.632493   
          Brand Group 43      0.000000      0.000000      0.000000   
          Brand Group 44    576.048693    871.090969   1361.650537   
          Brand Group 45   1995.503104   1663.363557   1307.480906   
          Brand Group 46   1335.533122   1394.229661   1505.275423   
          Brand Group 51    368.564296    400.115372    594.180684   
          Brand Group 58   1997.748581   1305.279925   1295.629396   
          Brand Group 63  -2408.708191    -69.668974    521.596040   
          Brand Group 70    553.832361   1916.631439    991.594098   
          Brand Group 71     40.007880     36.881432     74.881322   
          Brand Group 72    136.189910     68.737912    131.765769   
          Brand Group 73      0.000000      0.000000      0.000000   
...                                ...           ...           ...   
Cluster 9 Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 19   1065.970694    965.448115    826.204342   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1311.951469   1228.487730   1596.800828   
          Brand Group 26    578.979771    543.785442    501.128984   
          Brand Group 27    247.781298    204.107500    311.394846   
          Brand Group 30    849.946315    827.654128    931.822650   
          Brand Group 31     33.307956     37.928092     34.382406   
          Brand Group 36    411.032237    203.143857    326.407941   
          Brand Group 38     99.350845     45.914903     62.179995   
          Brand Group 39     49.729809     58.723182     58.563123   
          Brand Group 41  21390.272688  23210.413172  26803.499781   
          Brand Group 45    530.340734    465.930587    457.566091   
          Brand Group 46   2933.358538   2485.503539   3695.535976   
          Brand Group 51    109.127156    144.024336    126.068284   
          Brand Group 58    600.962268    495.003715    606.023913   
          Brand Group 70    123.079074    102.217856    118.692469   
          Brand Group 71    115.778040    101.534790    104.466731   
          Brand Group 72      1.273245      2.122076      1.697661   
          Brand Group 73      0.000000      0.000000      0.000000   
          Brand Group 76    395.554874    233.415453    362.856133   
          Brand Group 78     59.237114     50.176649     73.802040   
          Brand Group 79      1.564914      0.959141      1.682703   
          Brand Group 84   1223.608330   1416.356354   1882.021613   
          Brand Group 87   1350.142011  

In [465]:
dfpred=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value")

In [466]:
dfsales2.reset_index()

,Cluster,Brand Group,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,Jul 2012,Aug 2012,...,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,Cluster 1,Brand Group 12,616.408961,685.081035,968.938870,723.701936,656.607506,319.401731,953.191022,658.304347,...,1494.614561,1583.525099,1396.117515,1471.551885,1632.173591,1627.574308,1608.715411,1538.061964,1574.399619,1.381174e+03
1,Cluster 1,Brand Group 13,62.381470,69.445132,42.081080,55.139918,65.677360,36.194606,46.670999,62.483972,...,5.496543,14.480741,9.765432,11.746420,7.310123,10.965185,5.803457,5.608148,8.537778,6.361481e+00
2,Cluster 1,Brand Group 15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,613.388915,-229.544362,798.197908,-193.611296,180.088396,-126.788616,35.634313,177.382970,48.785139,-2.653626e+02
3,Cluster 1,Brand Group 16,2457.695992,2616.171336,2019.736343,2521.239601,2341.879305,1909.803910,2335.718836,2284.694096,...,556.856275,516.432602,711.261828,575.720936,638.983822,590.373449,612.722539,535.401404,566.573007,5.391845e+02
4,Cluster 1,Brand Group 17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1632.612631,1990.658124,1586.519705,1959.033915,2334.987199,2371.470463,2283.761079,2830.252855,2530.420686,2.870711e+03
5,Cluster 1,Brand Group 18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.704348,-5.247172,0.447933,3.171981,-68.883897,40.824615,-0.139374,0.011042,-23.587434,3.298240e+02
6,Cluster 1,Brand Group 19,1328.072408,1747.087079,1843.272270,2039.346028,1691.095349,2047.555687,1610.354841,1770.380294,...,893.457850,855.506720,802.064436,1066.648259,785.950844,886.596334,755.373756,959.422827,953.348555,6.881483e+02
7,Cluster 1,Brand Group 24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,535.500112,617.733001,744.317609,906.058486,837.815300,754.027434,1193.776122,1224.923058,1434.480782,1.314431e+03
8,Cluster 1,Brand Group 25,1869.355444,1724.577575,3067.466622,2478.010511,1714.321134,1882.068631,2453.273551,2329.189122,...,1425.465837,1150.809671,1356.505256,1254.493173,1282.958362,1269.700506,1429.508574,1275.457380,1358.765291,1.249301e+03
9,Cluster 1,Brand Group 26,2044.415276,2124.060822,2773.263487,2272.917754,2528.122320,2218.761723,2335.513978,2267.608240,...,2842.389283,2684.698994,2723.677431,2740.300859,2542.729046,2499.681858,2571.601476,2224.735039,2913.654991,2.441371e+03


In [467]:
dfindexes=dfsales2.reset_index()[["Cluster","Brand Group"]]

In [468]:
# GET LABELSSSSSSSSSSSSSSSs

In [469]:
dfsales2.columns

Index(['Jan 2012', 'Feb 2012', 'Mar 2012', 'Apr 2012', 'May 2012', 'Jun 2012',
       'Jul 2012', 'Aug 2012', 'Sep 2012', 'Oct 2012', 'Nov 2012', 'Dec 2012',
       'Jan 2013', 'Feb 2013', 'Mar 2013', 'Apr 2013', 'May 2013', 'Jun 2013',
       'Jul 2013', 'Aug 2013', 'Sep 2013', 'Oct 2013', 'Nov 2013', 'Dec 2013',
       'Jan 2014', 'Feb 2014', 'Mar 2014', 'Apr 2014', 'May 2014', 'Jun 2014',
       'Jul 2014', 'Aug 2014', 'Sep 2014', 'Oct 2014', 'Nov 2014', 'Dec 2014',
       'Jan 2015', 'Feb 2015', 'Mar 2015', 'Apr 2015', 'May 2015', 'Jun 2015',
       'Jul 2015', 'Aug 2015', 'Sep 2015', 'Oct 2015', 'Nov 2015', 'Dec 2015',
       'Jan 2016', 'Feb 2016', 'Mar 2016', 'Apr 2016', 'May 2016', 'Jun 2016',
       'Jul 2016', 'Aug 2016', 'Sep 2016', 'Oct 2016', 'Nov 2016', 'Dec 2016',
       'Jan 2017', 'Feb 2017', 'Mar 2017', 'Apr 2017', 'May 2017', 'Jun 2017',
       'Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017'],
      dtype='object')

In [470]:
#      'Jan 2017', 'Feb 2017', 'Mar 2017', 'Apr 2017', 'May 2017', 
dfpred=dfsales2.reset_index()[[
 'Jun 2017',
       'Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017']]

In [471]:
dfpred["Month"]=11

In [472]:
#       'Lag-11', 'Lag-10', 'Lag-9', 'Lag-8', 'Lag-7',
dfpred.columns=[
 'Lag-6', 'Lag-5', 'Lag-4',
       'Lag-3', 'Lag-2', 'Lag-1','Actual','Month']

In [473]:
dfpred['sales_trend'] = dfpred['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpred['sales_trend'] -= dfpred[ft_name]

dfpred['sales_trend'] /= len(lag_list) + 1

In [474]:
dfpred['mid_sales_trend'] = dfpred['Actual']

lag_list=[1,2,3,4,5,6]

for lag in lag_list:
    ft_name = ('Lag-%s' % lag)
    dfpred['mid_sales_trend'] -= dfpred[ft_name]

dfpred['mid_sales_trend'] /= len(lag_list) + 1

In [475]:
# dfpred['long_sales_trend'] = dfpred['Actual']

# lag_list=[1,2,3,4,5,6,7,8,9,10,11]

# for lag in lag_list:
#     ft_name = ('Lag-%s' % lag)
#     dfpred['long_sales_trend'] -= dfpred[ft_name]

# dfpred['long_sales_trend'] /= len(lag_list) + 1

In [476]:
dfpred.head(5)

,Lag-6,Lag-5,Lag-4,Lag-3,Lag-2,Lag-1,Actual,Month,sales_trend,mid_sales_trend
0,1471.551885,1632.173591,1627.574308,1608.715411,1538.061964,1574.399619,1381.174249,11,-835.000686,-1153.043218
1,11.746420,7.310123,10.965185,5.803457,5.608148,8.537778,6.361481,11,-3.396975,-6.229947
2,-193.611296,180.088396,-126.788616,35.634313,177.382970,48.785139,-265.362597,11,-131.791255,-55.264786
3,575.720936,638.983822,590.373449,612.722539,535.401404,566.573007,539.184516,11,-293.878109,-425.798663
4,1959.033915,2334.987199,2371.470463,2283.761079,2830.252855,2530.420686,2870.710849,11,-1193.430943,-1634.173621


In [477]:
#dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [478]:
def add_values(pred_x, new_pred,month):
    pred_x['Month']=month
    
#     pred_x["Lag-11"]=pred_x["Lag-10"]
#     pred_x["Lag-10"]=pred_x["Lag-9"]
#     pred_x["Lag-9"]=pred_x["Lag-8"]
#     pred_x["Lag-8"]=pred_x["Lag-7"]
#     pred_x["Lag-7"]=pred_x["Lag-6"]
    pred_x["Lag-6"]=pred_x["Lag-5"]
    pred_x["Lag-5"]=pred_x["Lag-4"]
    pred_x["Lag-4"]=pred_x["Lag-3"]
    pred_x["Lag-3"]=pred_x["Lag-2"]
    pred_x["Lag-2"]=pred_x["Lag-1"]
    pred_x["Lag-1"]=pred_x["Actual"]
    pred_x["Actual"]=new_pred

    pred_x['sales_trend'] = pred_x['Actual']
    lag_list=[1,2,3]

    for lag in [1,2,3]:
        ft_name = ('Lag-%s' % lag)
        pred_x['sales_trend'] -= pred_x[ft_name]

    pred_x['sales_trend'] /= len(lag_list) + 1
    
    pred_x['mid_sales_trend'] = pred_x['Actual']

    lag_list=[1,2,3,4,5,6]

    for lag in lag_list:
        ft_name = ('Lag-%s' % lag)
        pred_x['mid_sales_trend'] -= pred_x[ft_name]

    pred_x['mid_sales_trend'] /= len(lag_list) + 1
    
    #pred_x['long_sales_trend'] = dfpre['Actual']

#     lag_list=[1,2,3,4,5,6,7,8,9,10,11]

#     for lag in lag_list:
#         ft_name = ('Lag-%s' % lag)
#         pred_x['long_sales_trend'] -= dfpre[ft_name]

#     pred_x['long_sales_trend'] /= len(lag_list) + 1

    return pred_x

In [479]:
list_predictions=[]

In [480]:
# 'Lag-7', 'Lag-8','Lag-9', 'Lag-10', 'Lag-11',
xpred=dfpred[['Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'Lag-4', 'Lag-5', 'Lag-6', 'sales_trend','mid_sales_trend']]

In [481]:
for i in range(12):
    print(i)
    preds = [ m.predict(xpred) for m in models ]
    preds = np.mean(preds,axis=0)
    #print(preds)
    #preds=regres.predict(xpred)
    list_predictions.append(preds)
    xpred=add_values(xpred, preds,i)

0
1
2
3
4
5
6
7
8
9
10
11


In [482]:
len(list_predictions)

12

In [483]:
for d in range(len(list_predictions)):
    dfindexes['Month-%s' % d]=list_predictions[d]

In [484]:
dfindexes["Brand Group"].unique()

array(['Brand Group 12', 'Brand Group 13', 'Brand Group 15',
       'Brand Group 16', 'Brand Group 17', 'Brand Group 18',
       'Brand Group 19', 'Brand Group 24', 'Brand Group 25',
       'Brand Group 26', 'Brand Group 27', 'Brand Group 28',
       'Brand Group 30', 'Brand Group 31', 'Brand Group 36',
       'Brand Group 38', 'Brand Group 39', 'Brand Group 40',
       'Brand Group 41', 'Brand Group 43', 'Brand Group 44',
       'Brand Group 45', 'Brand Group 46', 'Brand Group 51',
       'Brand Group 58', 'Brand Group 63', 'Brand Group 70',
       'Brand Group 71', 'Brand Group 72', 'Brand Group 73',
       'Brand Group 76', 'Brand Group 78', 'Brand Group 79',
       'Brand Group 80', 'Brand Group 84', 'Brand Group 85',
       'Brand Group 87', 'Brand Group 89', 'Brand Group 90',
       'Brand Group 92', 'Brand Group 94', 'Brand Group 96',
       'Brand Group 97', 'Brand Group 20', 'Brand Group 98',
       'Brand Group 60', 'Brand Group 64', 'Brand Group 21'], dtype=object)

In [485]:
dfindexes

,Cluster,Brand Group,Month-0,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11
0,Cluster 1,Brand Group 12,1536.733675,1517.751661,1499.496211,1506.163276,1510.732505,1503.029528,1529.742785,1519.908548,1523.638631,1530.978558,1534.150847,1538.088101
1,Cluster 1,Brand Group 13,7.086026,7.058585,6.770347,6.828449,6.768794,6.685259,6.647840,6.587684,6.527929,6.474527,6.417775,6.362071
2,Cluster 1,Brand Group 15,-36.767035,-91.920556,-91.011674,-67.374106,-82.267449,-79.997419,-80.246023,-80.271021,-80.138514,-79.892631,-79.777306,-79.795611
3,Cluster 1,Brand Group 16,555.407247,551.168901,548.590765,545.817131,547.142419,544.286217,544.535360,542.917134,541.767240,540.829890,539.831962,538.685999
4,Cluster 1,Brand Group 17,2757.514377,2749.046490,2789.735923,2864.159590,2796.764836,2870.359684,2841.438539,2860.915085,2881.076045,2890.596310,2889.229857,2913.186515
5,Cluster 1,Brand Group 18,104.718739,146.015804,152.751274,133.102245,142.057055,147.207007,139.269363,142.578798,141.609583,140.503599,140.903921,140.380989
6,Cluster 1,Brand Group 19,827.149804,828.885695,803.715141,825.187760,807.090078,797.582726,815.652650,806.752569,805.092361,807.069001,803.763512,803.848711
7,Cluster 1,Brand Group 24,1284.381523,1326.445332,1368.824437,1346.221922,1359.258934,1343.913931,1354.166709,1363.075177,1363.434977,1362.456645,1367.522593,1368.685986
8,Cluster 1,Brand Group 25,1312.539210,1316.412394,1315.640096,1302.994770,1317.275353,1309.210670,1321.332322,1319.830304,1320.793410,1323.325981,1326.897634,1327.979882
9,Cluster 1,Brand Group 26,2514.633355,2577.918427,2570.470048,2559.354139,2653.289143,2556.918218,2621.682687,2635.204730,2633.327510,2648.222091,2667.086577,2661.596655


In [486]:
dfindexes.iloc[0][2:]

Month-0     1536.73
Month-1     1517.75
Month-2      1499.5
Month-3     1506.16
Month-4     1510.73
Month-5     1503.03
Month-6     1529.74
Month-7     1519.91
Month-8     1523.64
Month-9     1530.98
Month-10    1534.15
Month-11    1538.09
Name: 0, dtype: object

In [487]:
import numpy as np
dir_submission_template = 'Data_Novartis_Datathon-Results_Challenge1_Template.csv'

submission_template = pd.read_csv(dir_submission_template)

bg_set = set()

#a = pd.DataFrame(final_res, columns=['Cluster', 'Brand Group', 'res'])

for i, r in submission_template.iterrows():
    bg = r['Brand Group'][12:].replace(',', '').split(' ')
    bg_set.update(bg)
    bg = [ 'Brand Group ' + str(i) for i in bg ]
    print(r['Cluster'], r['Brand Group'], bg)
    
#    print(type(r['res']))
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'] in bg ]
#    res = res['res']
    res = np.sum(res, axis=0)
    submission_template.iloc[i, 2:] = res
    
    
for i, r in submission_template[submission_template['Brand Group'] == 'others'].iterrows():
    print(r.Cluster)
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res1 = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'][12:] not in bg_set ]

    res2 = np.sum(res1, axis=0)
    submission_template.iloc[i, 2:] = res2

Cluster 1 Brand Group 17 ['Brand Group 17']
Cluster 2 Brand Group 17 ['Brand Group 17']
Cluster 3 Brand Group 17 ['Brand Group 17']
Cluster 4 Brand Group 17 ['Brand Group 17']
Cluster 5 Brand Group 17 ['Brand Group 17']
Cluster 8 Brand Group 17 ['Brand Group 17']
Cluster 9 Brand Group 17 ['Brand Group 17']
Cluster 10 Brand Group 17 ['Brand Group 17']
Cluster 1 Brand Group 24 ['Brand Group 24']
Cluster 3 Brand Group 24 ['Brand Group 24']
Cluster 4 Brand Group 24 ['Brand Group 24']
Cluster 5 Brand Group 24 ['Brand Group 24']
Cluster 8 Brand Group 24 ['Brand Group 24']
Cluster 9 Brand Group 24 ['Brand Group 24']
Cluster 10 Brand Group 24 ['Brand Group 24']
Cluster 1 Brand Group 30 ['Brand Group 30']
Cluster 2 Brand Group 30 ['Brand Group 30']
Cluster 4 Brand Group 30 ['Brand Group 30']
Cluster 5 Brand Group 30 ['Brand Group 30']
Cluster 7 Brand Group 30 ['Brand Group 30']
Cluster 8 Brand Group 30 ['Brand Group 30']
Cluster 9 Brand Group 30 ['Brand Group 30']
Cluster 10 Brand Group 30 ['Br

In [488]:
submission_template.to_csv("submission_merged.csv",index=False)